In [1]:
# Insert code here.
import pandas as pd
import numpy as np
import random
import re
import time
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, BertConfig, AutoModel
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from torch.utils.data import Dataset
from tqdm import tqdm
# from sentence_transformers import SentenceTransformer
# sent_encoder = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
torch.cuda.empty_cache()

There are 4 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [3]:
models = ['ai4bharat/indic-bert','../temp/tapt-model']
model_num = 1
tokenizer = AutoTokenizer.from_pretrained(models[model_num])
labels = ['fake','hate', 'defamation','offensive','non-hostile']
lab_num = 3
src = '../temp/preprocessed/'

In [4]:
# train = pd.read_csv('../datasets/covid/Constraint_English_Train - Sheet1.csv')
# test = pd.read_csv('../datasets/covid/Constraint_English_Val - Sheet1.csv')

In [5]:
import pickle
with open(src+'train.pickle','rb') as f:
    train = pickle.load(f)
    train = pd.DataFrame.from_dict(train)
    train.drop(train.head(1).index, inplace=True)
with open(src+'valid.pickle','rb') as f:
    valid = pickle.load(f)
    valid = pd.DataFrame.from_dict(valid)
    valid.drop(valid.head(1).index, inplace=True)
with open(src+'test.pickle','rb') as f:
    test = pickle.load(f)
#     del test['task_1']
    test = pd.DataFrame.from_dict(test)
#     test.drop(test.head(1).index, inplace=True)
#     test = pd.DataFrame.from_dict(test)
# test = pd.read_csv('data/valid.tsv', sep='\t')

In [6]:
train = pd.concat([train, valid])
test.head(10)

,tweet_id,full_tweet,tweet_raw_text,hashtags,smiley,emoji,url,mentions,numerals,reserved_word,emotext,segmented_hash,clean
0,1,कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...,कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...,[],[],[],[],[],"[20, 6, 10, 20, 6]",[],[],[],कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...
1,2,पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...,पटना : BMP कैंप में पुरुष और महिला कांस्टेबल ...,[],[],[],[https://t.co/Dq05hREifM],[@kumarprakash4u],[],[],[],[],पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...
2,3,"कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...","कोई भी कांग्रेसी , ऊंची छत पर , रेलवे लाइन प...",[],[],"[🙏, 😂, 👍]",[],[],[],[],"[folded hands, face with tears of joy, thumbs up]",[],"कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ..."
3,4,अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...,अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...,[],[],[],[],[],[],[],[],[],अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...
4,5,RT @_Pb_swain_: इन पंचर छापों को कोन समझाए कि ...,: इन पंचर छापों को कोन समझाए कि उनके रोजगार म...,[],[],"[👇, 😂, 😂, 😂, 😂]",[],[@_Pb_swain_],[],[RT],"[backhand index pointing down, face with tears...",[],RT : इन पंचर छापों को कोन समझाए कि उनके रोजगा...
5,6,"पश्चिम बंगाल में फिर हुई माओवादियों की वापसी, ...","पश्चिम बंगाल में फिर हुई माओवादियों की वापसी ,...","[#Maoist, #WestBengal]",[],[],[https://t.co/pP1AOvOv0b],[],[],[],[],"[maoist, west bengal]","पश्चिम बंगाल में फिर हुई माओवादियों की वापसी, ..."
6,7,#Breaking-कंगना मामले पर बोले मनोज तिवारी-कहा ...,#Breaking-कंगना मामले पर बोले मनोज तिवारी-कहा ...,"[#Breaking, #Sushantsinghcase, #Kangana]",[],[],[https://t.co/szOTZWq1hI],[],[],[],[],"[breaking, sushantsinghcase, kangana]",-कंगना मामले पर बोले मनोज तिवारी-कहा धमकी मिल...
7,8,@BasudebaTripat4: @Rajanspsingh1 अच्छा किया सा...,": अच्छा किया साले का सर फोड़ दिया , , गर्दन...",[],[],[],[],"[@BasudebaTripat4, @Rajanspsingh1]",[],[],[],[],": अच्छा किया साले का सर फोड़ दिया,, गर्दन त..."
8,9,हैदराबाद बीजेपी विधायक राजा सिंह की बहन माया द...,हैदराबाद बीजेपी विधायक राजा सिंह की बहन माया द...,[],[],[],[],[],[],[],[],[],हैदराबाद बीजेपी विधायक राजा सिंह की बहन माया द...
9,10,"कमलनाथ के राज में 100,₹ में 100यूनिट बिजली मिल...","कमलनाथ के राज में , ₹ में 100यूनिट बिजली मिल र...",[],[],[],[],[],"[100, 100, 500, 1000]",[],[],[],"कमलनाथ के राज में 100,₹ में 100यूनिट बिजली मिल..."


In [7]:
lab = labels[lab_num]
# def label_encode(val):
#     return labels.index(val)
def label_encode(val):
    val = val.split(',')
    if lab_num == 4:
        if lab in val:
            return 0
        else:
            return 1
    else:
        if lab in val:
            return 1
        else:
            return 0

In [8]:
train['label'] = train.task_1.apply(label_encode)
train['tweet'] = train.full_tweet
test['tweet'] = test.full_tweet

In [9]:
train.label.sample(10)

1915    0
4       1
4605    0
2017    0
2340    0
573     0
4920    0
135     0
1878    0
589     0
Name: label, dtype: int64

In [10]:
train = train.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = []

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
# train.tweet = train.tweet.apply(clean_text)
# train.tweet = train.tweet.str.replace('\d+', '')

In [11]:
# test.label = test.label.apply(label_encode)
test = test.reset_index(drop=True)
# test.tweet = test.tweet.apply(clean_text)
# test.tweet = test.tweet.str.replace('\d+', '')

In [12]:
train.tweet.sample(10)

370     महाराष्ट्र के बाद अब मध्य प्रदेश में फ्लैट हुआ...
2173    शहीद वीरपाल सिंह जम्मू-कश्मीर के सुंदरी वन में...
933     सुशांत की ऑटोप्सी केस में सख्त MSHRC, कूपर अस्...
468     मैंने कांग्रेस को वोट दिया है जिस बात का डर था...
1546    पी के एक गांव में कोरोना स्क्रीनिंग करने गईं ड...
983     इलाहाबाद हाईकोर्ट ने दिया डॉ कफील खान की तुरंत...
4224    कम्युनिष्टों की हरकत देखो: शाहीन बाग के इवेंट ...
2734    अंतरराष्ट्रीय उड़ानों के माध्यम से वाराणसी, लख...
4785    पुलिस विभाग का भी निजिकरण होना चाहिए   क्योंकि...
4227     सुरेश रैना के बाद हरभजन सिंह ने भी आईपीएल 202...
Name: tweet, dtype: object

In [13]:
# split the dataset into training and validation datasets 
from sklearn.model_selection import train_test_split
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['tweet'], train['label'])
train_x, valid_x, train_y, valid_y = train_test_split(train['tweet'], train['label'], test_size=0.2)

In [14]:
def count_words(text):
    try:
        return len(text.split())
    except:
        print(text)
        return None

In [15]:
total = 0
maxw = 0
large_count = 0
for i in train_x:
    temp = count_words(i)
    total += temp
    maxw = temp if temp > maxw else maxw
    large_count += 1 if temp > 120 else 0
total/len(train_x), maxw, large_count, len(train_x)

(30.687549721559268, 1990, 26, 5028)

In [16]:
# MAX_LENGTH = 50
posts = train.values
categories = train.values

In [17]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 1e-05


In [18]:
import gensim.models as gsm
e2v = gsm.KeyedVectors.load_word2vec_format('emoji2vec.bin', binary=True)
# happy_vector = e2v['😂']    # Produces an embedding vector of length 300

# Download the bin file from here https://github.com/uclnlp/emoji2vec/blob/master/pre-trained/emoji2vec.bin

def getEmojiEmbeddings(emojiList,dim=300,verbose = False):
  """ Generates an emoji vector by averaging the emoji representation for each emoji. If no emoji returns an empty list of dimension dim"""
  if dim < 300:
    raise IndexError("Dim has to be greater than 300")
  result = np.zeros(dim)
  if (len(emojiList) == 0):
    return result
  else:
    embs = None
    for i in emojiList:
      if verbose:
        if i not in e2v.vocab:
          print(i)
    embs = np.mean([e2v[i] for i in emojiList if i in e2v.vocab], axis=0)
  if np.any(np.isnan(embs)):
    return result
  result[:300] = embs
  return result
getEmojiEmbeddings(valid.emoji.values[0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [19]:
ids = tokenizer.encode_plus(
            valid.full_tweet.values[0],
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=128,
            pad_to_max_length=True,
            return_attention_mask = True,
            return_token_type_ids=True
        )['input_ids']
torch.tensor(ids, dtype=torch.long).shape, torch.tensor(getEmojiEmbeddings(valid.emoji.values[0]), dtype=torch.long).shape

/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(torch.Size([128]), torch.Size([300]))

In [20]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len, t = False):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.tweet
        self.emoji = dataframe.emoji
        self.hash = dataframe.segmented_hash
        self.t = t
        if not self.t:
            self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_attention_mask = True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        h_text = self.hash[index]
        h_text = " ".join(h_text)
        inputs = self.tokenizer.encode_plus(
            h_text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_attention_mask = True,
            return_token_type_ids=True
        )
        h_ids = inputs['input_ids']
        h_mask = inputs['attention_mask']
        h_token_type_ids = inputs["token_type_ids"]
#         h_inputs
        emoji = getEmojiEmbeddings(self.emoji[index])
        if self.t:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'h_ids': torch.tensor(h_ids, dtype=torch.long),
                'h_mask': torch.tensor(h_mask, dtype=torch.long),
                'h_token_type_ids': torch.tensor(h_token_type_ids, dtype=torch.long),
                'emoji' : torch.tensor(emoji, dtype=torch.long),
            }
        else:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'h_ids': torch.tensor(h_ids, dtype=torch.long),
                'h_mask': torch.tensor(h_mask, dtype=torch.long),
                'h_token_type_ids': torch.tensor(h_token_type_ids, dtype=torch.long),
                'emoji' : torch.tensor(emoji, dtype=torch.long),
                'targets': torch.tensor(self.targets[index], dtype=torch.long)
            }

In [21]:
# Creating the dataset and dataloader for the neural network

train_size = 0.85
train_data=train.sample(frac=train_size,random_state=200)
test_data=train.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (6286, 16)
TRAIN Dataset: (5343, 16)
TEST Dataset: (943, 16)


In [22]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [23]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(models[model_num])
        self.l2 = AutoModel.from_pretrained(models[model_num])
        
        self.pre_classifier_1 = torch.nn.Linear(768, 768)
        self.pre_classifier_2 = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.pre_classifier_3 = torch.nn.Linear(1836, 1836)
#         self.pre_classifier_3 = torch.nn.Linear(768, 100)
        self.classifier = torch.nn.Linear(1836, 2)

    def forward(self, input_ids, attention_mask, token_type_ids, h_ids, h_mask, h_token_type_ids, emoji):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state_1 = output_1[0]
        pooler_1 = hidden_state_1[:, 0]
        pooler_1 = self.pre_classifier_1(pooler_1)
        pooler_1 = torch.nn.Tanh()(pooler_1)
        pooler_1 = self.dropout(pooler_1)
        output_2 = self.l2(input_ids=h_ids, attention_mask=h_mask)
        hidden_state_2 = output_2[0]
        pooler_2 = hidden_state_2[:, 0]
        pooler_2 = self.pre_classifier_2(pooler_2)
        pooler_2 = torch.nn.Tanh()(pooler_2)
        pooler_2 = self.dropout(pooler_2)
        pooler_3 = torch.cat((pooler_1, pooler_2), 1)
        pooler_3 = torch.cat((pooler_3, emoji), 1)
#         print(pooler_1.shape,hidden_state_1.shape, pooler_2.shape, emoji.type(torch.FloatTensor).shape)
#         pooler_3 = torch.nn.Tanh()(emoji.type(torch.FloatTensor))
#         pooler_3 = self.dropout(pooler_3)
#         print(pooler_3.shape)
        pooler_3 = self.pre_classifier_3(pooler_3)
#         pooler_3 = self.pre_classifier_3(pooler_2)
        pooler_3 = torch.nn.Tanh()(pooler_3)
        pooler_3 = self.dropout(pooler_3)
        output = self.classifier(pooler_3)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
             

In [24]:
# from torchsummary import summary
# print(repr(model))


In [25]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


The BERT model has 58 different named parameters.

==== Embedding Layer ====

l1.embeddings.word_embeddings.weight                    (200000, 128)
l1.embeddings.position_embeddings.weight                  (512, 128)
l1.embeddings.token_type_embeddings.weight                  (2, 128)
l1.embeddings.LayerNorm.weight                                (128,)
l1.embeddings.LayerNorm.bias                                  (128,)

==== First Transformer ====

l1.encoder.embedding_hidden_mapping_in.weight             (768, 128)
l1.encoder.embedding_hidden_mapping_in.bias                   (768,)
l1.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight       (768,)
l1.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias       (768,)
l1.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight   (768, 768)
l1.encoder.albert_layer_groups.0.albert_layers.0.attention.query.bias       (768,)
l1.encoder.albert_layer_groups.0.albert_layers.0.attention.k

In [26]:
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

In [27]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [28]:
def train(epoch):
    total_train_loss = 0
    count = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        h_ids = data['h_ids'].to(device, dtype = torch.long)
        h_mask = data['h_mask'].to(device, dtype = torch.long)
        h_token_type_ids = data['h_token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        emoji = data['emoji'].to(device, dtype = torch.long)
        outputs = model(ids, mask, token_type_ids, h_ids, h_mask, h_token_type_ids, emoji)
        optimizer.zero_grad()
#         loss = outputs.loss
        loss = loss_fn(outputs, targets)
#         if _%50==0:
#             print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        total_train_loss += loss.item()
        count += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    print(f'Epoch: {epoch}, Loss:  {total_train_loss/count}')
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            h_ids = data['h_ids'].to(device, dtype = torch.long)
            h_mask = data['h_mask'].to(device, dtype = torch.long)
            h_token_type_ids = data['h_token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            emoji = data['emoji'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids, h_ids, h_mask, h_token_type_ids, emoji)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    fin_outputs = list(np.argmax(np.array(fin_outputs), axis=1).flatten())
    print(classification_report(fin_targets, fin_outputs))
    torch.save(model, '../temp/finetuned/'+lab+'_epoch_'+str(epoch))
    return fin_outputs, fin_targets
#     final_outputs = np.array(fin_outputs) >=0.5
#     final = []
#     final_t = []
#     final_fine = [[],[],[],[]]
#     final_fine_t = [[],[],[],[]]
#     for (i,j) in zip(final_outputs, fin_targets):
#         output_sum = sum(i)
#         target_sum = sum(j)
#         if output_sum == 0:
#             final.append(0)
#         else:
#             final.append(1)
#         if target_sum == 0:
#             final_t.append(0)
#         else:
#             final_t.append(1)
#         for p in range(4):
#             final_fine[p].append(int(i[p]))
#             final_fine_t[p].append(int(j[p]))
#     print("Coarse:")
#     print(classification_report(final, final_t))
#     for i in range(4):
#         print("Fine", i)
    
#     return fin_outputs, fin_targets

In [29]:
for epoch in range(EPOCHS):
    out, tar = train(epoch)
#     break

3it [00:00,  4.05it/s]/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
334it [01:21,  4.08it/s]
1it [00:00,  5.49it/s]

Epoch: 0, Loss:  0.35052143896143595


30it [00:05,  5.60it/s]
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

              precision    recall  f1-score   support

           0       0.86      1.00      0.93       812
           1       0.00      0.00      0.00       131

    accuracy                           0.86       943
   macro avg       0.43      0.50      0.46       943
weighted avg       0.74      0.86      0.80       943

